# 加载数据

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from torchvision.transforms import transforms
from torchsummary import summary

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Oct 15 11:08:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Data.zip


流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: Data/data/aug_train/796_Annotation.png  
  inflating: Data/data/aug_train/797.png  
  inflating: Data/data/aug_train/797_11.png  
  inflating: Data/data/aug_train/797_11_Annotation.png  
  inflating: Data/data/aug_train/797_12.png  
  inflating: Data/data/aug_train/797_12_Annotation.png  
  inflating: Data/data/aug_train/797_13.png  
  inflating: Data/data/aug_train/797_13_Annotation.png  
  inflating: Data/data/aug_train/797_21.png  
 extracting: Data/data/aug_train/797_21_Annotation.png  
  inflating: Data/data/aug_train/797_22.png  
 extracting: Data/data/aug_train/797_22_Annotation.png  
  inflating: Data/data/aug_train/797_23.png  
 extracting: Data/data/aug_train/797_23_Annotation.png  
  inflating: Data/data/aug_train/797_31.png  
  inflating: Data/data/aug_train/797_31_Annotation.png  
  inflating: Data/data/aug_train/797_32.png  
  inflating: Data/data/aug_train/797_32_Annotation.png  
  inflating: Data/data/aug_train/797_33.png  
  infl

In [ ]:
!ls /content/Data/data/train -l | grep "^-" | wc -l
!ls /content/Data/data/test -l | grep "^-" | wc -l

1534
384


In [ ]:
%cd /content
if os.path.exists("Ultrasound"):
    %rm -rf Ultrasound
!ls
if not os.path.exists("Ultrasound"):
    !git clone https://github.com/z872845991/Ultrasound.git
%cd Ultrasound
!ls

/content
Data  drive  sample_data
Cloning into 'Ultrasound'...
remote: Enumerating objects: 1392, done.
remote: Counting objects: 100% (1392/1392), done.
remote: Compressing objects: 100% (1209/1209), done.
remote: Total 1392 (delta 205), reused 1338 (delta 151), pack-reused 0
Receiving objects: 100% (1392/1392), 28.01 MiB | 37.06 MiB/s, done.
Resolving deltas: 100% (205/205), done.
/content/Ultrasound
dataset		 model	tools  unet_res_myself_pool_connect_o_C.ipynb
main_unet.ipynb  runs	train  unet-visualize.py


In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# 参数

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from torchvision.transforms import transforms
from dataset.Fetus import FetusDataset
from model.unet import Unet
from tools.metrics import dice_coef,iou_score, get_accuracy, get_precision, get_specificity, get_recall, get_F1
from tools.utils import AverageMeter
import datetime

In [ ]:
num_epoch=81
batch_size={'train':2,
            'val':1
            }
train_path='/content/Data/data/train'
test_path='/content/Data/data/test'
#变换
x_transforms = transforms.Compose([
    transforms.Resize((512,512)),
    # transforms.CenterCrop(512),
    transforms.ToTensor()
    # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
# mask只需要转换为tensor
y_transforms = transforms.Compose([
    transforms.Resize((512, 512)),
    # transforms.CenterCrop(512),
    transforms.ToTensor()
   ])
#加载数据集



In [ ]:
def train_model(model, criterion, optimizer, dataload, num_epochs):
    # 这个是用来找到miou最好的一次epoch
    bigiou=0

    model.train()
    for epoch in range(num_epochs):
        # print(epoch)
        dt_size = len(dataload.dataset)
        epoch_loss = 0
        avgmeter1 = AverageMeter()
        avgmeter2 = AverageMeter()
        avgmeter3 = AverageMeter()
        avgmeter4 = AverageMeter()
        avgmeter5 = AverageMeter()
        avgmeter6 = AverageMeter()
        avgmeter7 = AverageMeter()

        step = 0

        for x, y, _ in dataload:
            step += 1
            inputs = x.cuda()
            labels = y.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)

            iou = iou_score(outputs, labels)
            dice = dice_coef(outputs, labels)
            ACC = get_accuracy(outputs, labels)
            PPV = get_precision(outputs, labels)
            TNR = get_specificity(outputs, labels)
            TPR = get_recall(outputs, labels)
            F1 = get_F1(outputs, labels)

            avgmeter1.update(iou, batch_size['train'])
            avgmeter2.update(dice, batch_size['train'])
            avgmeter3.update(ACC, batch_size['train'])
            avgmeter4.update(PPV, batch_size['train'])
            avgmeter5.update(TNR, batch_size['train'])
            avgmeter6.update(TPR, batch_size['train'])
            avgmeter7.update(F1, batch_size['train'])

            epoch_loss += loss.item()

            loss.backward()
            optimizer.step()


        with open('/content/drive/MyDrive/result/train/train_7m_unet.txt', 'a+') as file:
            file.write("epoch %d loss:%0.3f miou:%.3f maxiou:%.3f miniou:%.3f  mdice:%.3f maxdice:%.3f mindice:%.3f" % (
            epoch, epoch_loss / step, avgmeter1.avg, avgmeter1.max, avgmeter1.min, avgmeter2.avg, avgmeter2.max,
            avgmeter2.min) + '\n')

        test_model(epoch, model, bigiou)

        # torch.save(model.state_dict(), '/home/p920/cf/checkpoints/weights_r2unet_%d.pth' % epoch)

def test_model(epoch, model, bigiou):
    threshold = 0.5

    model.eval()
    test_dataset = FetusDataset(test_path, mode='train',transform=x_transforms,target_transform=y_transforms)
    dataloaders = DataLoader(test_dataset, batch_size=1)
    step = 0

    te_avgmeter1 = AverageMeter()
    te_avgmeter2 = AverageMeter()
    te_avgmeter3 = AverageMeter()
    te_avgmeter4 = AverageMeter()
    te_avgmeter5 = AverageMeter()
    te_avgmeter6 = AverageMeter()
    te_avgmeter7 = AverageMeter()


    for x, y, z in dataloaders:
        step += 1
        inputs = x.cuda()
        labels = y.cuda()
        outputs = model(inputs)


        iou1 = iou_score(outputs, labels)
        dice1 = dice_coef(outputs, labels)
        ACC1 = get_accuracy(outputs, labels)
        PPV1 = get_precision(outputs, labels)
        TNR1 = get_specificity(outputs, labels)
        TPR1 = get_recall(outputs, labels)
        F11 = get_F1(outputs, labels)

        te_avgmeter1.update(iou1)
        te_avgmeter2.update(dice1)
        te_avgmeter3.update(ACC1)
        te_avgmeter4.update(PPV1)
        te_avgmeter5.update(TNR1)
        te_avgmeter6.update(TPR1)
        te_avgmeter7.update(F11)


        if epoch == 80 and iou1 < threshold:
            with open('/content/drive/MyDrive/result/hard/hard_seg_7m_unet.txt', 'a+') as file:
                for s in z:
                    file.write(s + '\n')


    with open('/content/drive/MyDrive/result/test/test_7m_unet.txt', 'a+') as file:
        file.write(" ACC:%.4f  PPV:%.4f  TNR:%.4f  TPR:%.4f  F1:%.4f  miou:%.4f maxiou:%.4f miniou:%.4f  mdice:%.4f maxdice:%.4f mindice:%.4f iou1:%.4f iou2:%.4f iou3:%.4f iou4:%.4f iou5:%.4f iou6:%.4f iou7:%.4f iou8:%.4f dice1:%.4f dice2:%.4f dice3:%.4f dice4:%.4f dice5:%.4f dice6:%.4f dice7:%.4f dice8:%.4f" % (
            te_avgmeter3.avg, te_avgmeter4.avg, te_avgmeter5.avg, te_avgmeter6.avg, te_avgmeter7.avg, te_avgmeter1.avg, te_avgmeter1.max, te_avgmeter1.min, te_avgmeter2.avg, te_avgmeter2.max,te_avgmeter2.min, te_avgmeter1.first, te_avgmeter1.second, te_avgmeter1.third, te_avgmeter1.forth, te_avgmeter1.fifth, te_avgmeter1.sixth, te_avgmeter1.seventh, te_avgmeter1.eighth, te_avgmeter2.first, te_avgmeter2.second, te_avgmeter2.third, te_avgmeter2.forth, te_avgmeter2.fifth, te_avgmeter2.sixth, te_avgmeter2.seventh, te_avgmeter2.eighth) + '\n')


    if  te_avgmeter1.avg > bigiou:
        bigiou = te_avgmeter1.avg
        torch.save(model.state_dict(), '/content/drive/MyDrive/checkpoints/weights_7m_unet_%d.pth' % epoch)

In [ ]:
model=Unet(3,1)
model=model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())
fetus_dataset = FetusDataset(train_path, transform=x_transforms,target_transform=y_transforms)
dataloaders = DataLoader(fetus_dataset, batch_size=batch_size['train'], shuffle=True, num_workers=4,drop_last=False)
train_model(model, criterion, optimizer, dataloaders,num_epoch)